In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# configuração básica do driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

try:
    url = "https://www.docvirt.com/docreader.net/DocReader.aspx?bib=coportinari&id=4645702686696&pagfis=1"
    driver.get(url)

    # Aguarda o carregamento do AJAX (o site é lento devido ao framework Telerik)
    time.sleep(0.5)

    # Utilizando o find_element 
    # O primeiro argumento é a estratégia (By.ID), o segundo é o nome do ID
    textarea_element = driver.find_element(By.ID, "TextoDigitadoTxt")
    conteudo = textarea_element.get_attribute("value")

    pasta_element = driver.find_element(By.ID, "PastaTxt")
    texto_pasta = pasta_element.text  # Ou pasta_element.get_attribute("innerText")


    if conteudo:
        print("Texto encontrado:")
        print(conteudo)
        print("Texto da pasta:")
        print(texto_pasta)

    else:
        print("O elemento foi encontrado, mas ainda está vazio. Tente aumentar o time.sleep.")

finally:
    driver.quit()

Texto encontrado:
COD_TIPO: CO
NUM_ANTDOC: 1
PAGINAÇÃO: 
TIPO CORRESPONDÊNCIA: Telegrama
REMETENTE: Iris Abbott
ONOMÁSTICO: null
DESTINATARIO: Candido Portinari
MEMO_RESUMO: Convida Portinari para jantar, destacando presença do pintor Tanguy.
DATA_INICIO: 1940/11/08
Texto da pasta:
(CO) Consulta Correspondência\CO_0001


In [ ]:
def extract_text_from_page(url):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    try:
        driver.get(url)
        time.sleep(0.5)
        textarea_element = driver.find_element(By.ID, "TextoDigitadoTxt")
        conteudo = textarea_element.get_attribute("value")

        pasta_element = driver.find_element(By.ID, "PastaTxt")
        texto_pasta = pasta_element.text  

        linha_df = {
            "titulo": texto_pasta,
            "conteudo": conteudo
        }

        return linha_df
    
    except Exception as e:
        print(f"Erro ao extrair dados da página")
        return {"titulo": None, "conteudo": None}

    finally:
        driver.quit()

In [87]:
import pandas as pd
df_dados = pd.DataFrame(columns=["titulo", "conteudo"])

In [88]:
for i in range(1, 10):
    url = f"https://www.docvirt.com/docreader.net/DocReader.aspx?bib=coportinari&id=4645702686696&pagfis={i}"
    linha_df = extract_text_from_page(url)
    df_dados = pd.concat([df_dados, pd.DataFrame([linha_df])], ignore_index=True)
    

In [89]:
df_dados

,titulo,conteudo
0,(CO) Consulta Correspondência\CO_0001,COD_TIPO: CO\nNUM_ANTDOC: 1\nPAGINAÇÃO: \nTIPO...
1,(CO) Consulta Correspondência\CO_0002,COD_TIPO: CO\nNUM_ANTDOC: 2\nPAGINAÇÃO: \nTIPO...
2,(CO) Consulta Correspondência\CO_0003,COD_TIPO: CO\nNUM_ANTDOC: 3\nPAGINAÇÃO: \nTIPO...
3,(CO) Consulta Correspondência\CO_0004,COD_TIPO: CO\nNUM_ANTDOC: 4\nPAGINAÇÃO: \nTIPO...
4,(CO) Consulta Correspondência\CO_0005,COD_TIPO: CO\nNUM_ANTDOC: 5\nPAGINAÇÃO: \nTIPO...
5,(CO) Consulta Correspondência\CO_0006,COD_TIPO: CO\nNUM_ANTDOC: 6\nPAGINAÇÃO: \nTIPO...
6,(CO) Consulta Correspondência\CO_0007,COD_TIPO: CO\nNUM_ANTDOC: 7\nPAGINAÇÃO: \nTIPO...
7,(CO) Consulta Correspondência\CO_0007,
8,(CO) Consulta Correspondência\CO_0008,COD_TIPO: CO\nNUM_ANTDOC: 8\nPAGINAÇÃO: \nTIPO...


In [90]:
df_dados.to_csv("dados_extraidos.csv", index=False)